<a href="https://colab.research.google.com/github/fralfaro/MAT306/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT306 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [82]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT306/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [83]:
#a)
df_anio = pd.concat(
    [pd.read_csv(file) for file in archivos_anio],
    ignore_index=True)

df_anio.columns = df_anio.columns.str.lower()


In [84]:
df_anio

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.50,59.0
1,AGO,2001,30.20,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN
...,...,...,...,...
3055,WSM,2019,18.25,22.0
3056,YEM,2019,61.66,168.0
3057,ZAF,2019,22.19,31.0
3058,ZMB,2019,36.38,119.0


In [85]:
df_codigos

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos
...,...,...
176,YEM,Yemen
177,ZAF,Sudáfrica
178,ZMB,Zambia
179,ZWE,Zimbabue


In [86]:
#b)
duplicado_iso = df_codigos[df_codigos.duplicated(subset=['codigo_iso'], keep=False)]
print(duplicado_iso)


    codigo_iso      pais
179        ZWE  Zimbabue
180        ZWE      malo


In [87]:
df_codigos.drop(df_codigos[df_codigos['pais'] == 'malo'].index, inplace=True)

In [88]:
df_codigos

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos
...,...,...
175,WSM,Samoa
176,YEM,Yemen
177,ZAF,Sudáfrica
178,ZMB,Zambia


In [89]:
#c)
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')

In [90]:
df

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.50,59.0,Afghanistán
1,AGO,2001,30.20,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos
...,...,...,...,...,...
3055,WSM,2019,18.25,22.0,Samoa
3056,YEM,2019,61.66,168.0,Yemen
3057,ZAF,2019,22.19,31.0,Sudáfrica
3058,ZMB,2019,36.38,119.0,Zambia




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [91]:
#Estructura del DataFrame

print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
print(f"Nombre de columnas: {list(df.columns)}")
print("\nTipos de datos:")
print(df.dtypes)

Filas: 3060, Columnas: 5
Nombre de columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object


Así que tiene 3060 filas y 5 columnas cuyos nombres son 'codigo_iso', 'anio', 'indice', 'ranking' y 'pais', los tipos de datos de cada una de estas estan anotados arriba. Sí hay un tipo de dato extraño que es object que lo tiene 'pais' y 'codigo_iso'

In [92]:
#Resumen estadístico
print(df.describe())

print("---")

pais_min_indice = df.loc[df['indice'].idxmin(), 'pais']
pais_max_indice = df.loc[df['indice'].idxmax(), 'pais']
pais_min_ranking = df.loc[df['ranking'].idxmin(), 'pais']
pais_max_ranking = df.loc[df['ranking'].idxmax(), 'pais']

print(f"País con menor índice (mejor libertad): {pais_min_indice}")
print(f"País con mayor índice (peor libertad): {pais_max_indice}")
print(f"País con mejor ranking: {pais_min_ranking}")
print(f"País con peor ranking: {pais_max_ranking}")

              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.295000      34.000000
50%    2009.000000     28.000000      70.000000
75%    2015.000000     41.227500     110.000000
max    2019.000000  64536.000000  121056.000000
---
País con menor índice (mejor libertad): Dinamarca
País con mayor índice (peor libertad): Kosovo
País con mejor ranking: Finlandia
País con peor ranking: Kosovo


Podemos notar que la libertad de prensa cambia bastante entre paises pues std de indice es 2700 aproximadamente. El valor mínimo de indice es 0, el máximo es 64536 y el promedio es de 206 aproximadamente. Los países con los valores extremos estan anotados en la celda anterior

In [93]:
#Datos faltantes

#¿Cuántos valores nulos hay en cada columna?
null_counts = df.isnull().sum()
print(f"Valores nulos por columna:\n{null_counts}")

#¿Qué proporción de observaciones tienen valores faltantes?
null_proportions = df.isnull().mean()
print(f"\nProporción de valores nulos:\n{null_proportions}")

#¿Hay columnas con más del 30% de datos faltantes?
high_null_cols = null_proportions[null_proportions > 0.3]
print(f"\nColumnas con más del 30% de datos faltantes: {list(high_null_cols.index) if len(high_null_cols) > 0 else 'Ninguna'}")

Valores nulos por columna:
codigo_iso      0
anio            0
indice        396
ranking       223
pais            0
dtype: int64

Proporción de valores nulos:
codigo_iso    0.000000
anio          0.000000
indice        0.129412
ranking       0.072876
pais          0.000000
dtype: float64

Columnas con más del 30% de datos faltantes: Ninguna


In [94]:
#Unicidd y duplicados

print(f"Países distintos: {df['pais'].nunique()}")
print(f"Años distintos: {df['anio'].nunique()}")
print(f"Filas duplicadas: {df.duplicated().sum()}")

Países distintos: 179
Años distintos: 17
Filas duplicadas: 0


In [96]:
#Validación cruzada de columnas
inconsistencias = df.groupby('codigo_iso')['pais'].nunique()
inconsistencias = inconsistencias[inconsistencias > 1]
print(f"\nInconsistencias entre código ISO y país: {len(inconsistencias)}")
if len(inconsistencias) > 0:
    print(inconsistencias)


Inconsistencias entre código ISO y país: 0





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [97]:
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)]

In [98]:
df_america

,codigo_iso,anio,indice,ranking,pais
5,ARG,2001,12.00,8.0,Argentina
7,ATG,2001,NaN,NaN,Antigua y Barbuda
20,BLZ,2001,NaN,NaN,Belize
21,BOL,2001,14.50,13.0,Bolivia
22,BRA,2001,18.80,18.0,Brasil
...,...,...,...,...,...
3029,SUR,2019,16.38,20.0,Surinam
3043,TTO,2019,24.74,39.0,Trinidad y Tobago
3050,URY,2019,16.06,19.0,Uruguay
3051,USA,2019,25.69,48.0,Estados Unidos


In [99]:
#a)

#hay datos nulos asi que vamos verificando esto
for year in sorted(df_america['anio'].unique()):
    year_data = df_america[df_america['anio'] == year]


    if year_data.empty or year_data['indice'].isnull().all():
        print(f"Año {year}: No hay datos disponibles")
        continue


    min_idx_val = year_data['indice'].idxmin()
    max_idx_val = year_data['indice'].idxmax()


    if pd.isna(min_idx_val) or pd.isna(max_idx_val):
        print(f"Año {year}: Datos insuficientes para determinar extremos")
        continue

    min_idx = year_data.loc[min_idx_val]
    max_idx = year_data.loc[max_idx_val]

    print(f"Año {year}: Mejor - {min_idx['pais']} ({min_idx['indice']}), Peor - {max_idx['pais']} ({max_idx['indice']})")

Año 2001: Mejor - Canadá (0.8), Peor - Cuba (90.3)
Año 2002: Mejor - Trinidad y Tobago (1.0), Peor - Cuba (97.83)
Año 2003: Mejor - Trinidad y Tobago (2.0), Peor - Argentina (35826.0)
Año 2004: Mejor - Trinidad y Tobago (2.0), Peor - Cuba (87.0)
Año 2005: Mejor - Bolivia (4.5), Peor - Cuba (95.0)
Año 2006: Mejor - Canadá (4.88), Peor - Cuba (96.17)
Año 2007: Mejor - Canadá (3.33), Peor - Cuba (88.33)
Año 2008: Mejor - Canadá (3.7), Peor - Cuba (94.0)
Año 2009: Mejor - Estados Unidos (6.75), Peor - Cuba (78.0)
Año 2012: Mejor - Jamaica (9.88), Peor - Cuba (71.64)
Año 2013: Mejor - Jamaica (10.9), Peor - Cuba (70.92)
Año 2014: Mejor - Canadá (10.99), Peor - Cuba (70.21)
Año 2015: Mejor - Costa Rica (11.1), Peor - Cuba (70.23)
Año 2016: No hay datos disponibles
Año 2017: Mejor - Costa Rica (11.93), Peor - Cuba (71.75)
Año 2018: Mejor - Jamaica (11.33), Peor - Cuba (68.9)
Año 2019: Mejor - Jamaica (11.13), Peor - Cuba (63.81)


El indice de argentina del 2003 se ve bastante irreal no sé si habre cometido un error y cambiado datos de alguna forma o la base de datos es curiosa

In [100]:
#b)

valid_years = df_america.groupby('anio')['indice'].filter(lambda x: x.notnull().any())
df_valid = df_america[df_america.index.isin(valid_years.index)]

min_values = df_valid.loc[df_valid.groupby('anio')['indice'].idxmin()][['anio', 'pais', 'indice']]
max_values = df_valid.loc[df_valid.groupby('anio')['indice'].idxmax()][['anio', 'pais', 'indice']]

result = pd.merge(min_values, max_values, on='anio', suffixes=('_mejor', '_peor'))
print(result)

all_years = df_america['anio'].unique()
valid_year_list = df_valid['anio'].unique()
missing_years = set(all_years) - set(valid_year_list)

    anio         pais_mejor  indice_mejor  pais_peor  indice_peor
0   2001             Canadá          0.80       Cuba        90.30
1   2002  Trinidad y Tobago          1.00       Cuba        97.83
2   2003  Trinidad y Tobago          2.00  Argentina     35826.00
3   2004  Trinidad y Tobago          2.00       Cuba        87.00
4   2005            Bolivia          4.50       Cuba        95.00
5   2006             Canadá          4.88       Cuba        96.17
6   2007             Canadá          3.33       Cuba        88.33
7   2008             Canadá          3.70       Cuba        94.00
8   2009     Estados Unidos          6.75       Cuba        78.00
9   2012            Jamaica          9.88       Cuba        71.64
10  2013            Jamaica         10.90       Cuba        70.92
11  2014             Canadá         10.99       Cuba        70.21
12  2015         Costa Rica         11.10       Cuba        70.23
13  2017         Costa Rica         11.93       Cuba        71.75
14  2018  

### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [101]:
pivot_table = pd.pivot_table(df, values='indice', index='pais', columns='anio',
                            aggfunc='max', fill_value=0)
print(pivot_table.head())

anio         2001   2002   2003   2004   2005   2006   2007   2008   2009  \
pais                                                                        
Afghanistán  35.5  40.17  28.25  39.17  44.25  56.50  59.25  54.25  51.67   
Albania       0.0   6.50  11.50  14.17  18.00  25.50  16.00  21.75  21.50   
Alemania      1.5   1.33   2.00   4.00   5.50   5.75   4.50   3.50   4.25   
Algeria      31.0  33.00  43.50  40.33  40.00  40.50  31.33  49.56  47.33   
Andorra       0.0   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   

anio          2012   2013   2014   2015   2017   2018   2019  
pais                                                          
Afghanistán  37.36  37.07  37.44  37.75  39.46  37.28  36.55  
Albania      30.88  29.92  28.77  29.92  29.92  29.49  29.84  
Alemania     10.24  10.23  11.47  14.80  14.97  14.39  14.60  
Algeria      36.54  36.26  36.63  41.69  42.83  43.13  45.75  
Andorra       6.82   6.82  19.87  19.87  21.03  22.21  24.63  


In [102]:
#a)
paismax = pivot_table.max(axis=1).idxmax()
valormax = pivot_table.max(axis=1).max()
paismin = pivot_table.replace(0, np.nan).min(axis=1).idxmin()
valormin = pivot_table.replace(0, np.nan).min(axis=1).min()

print(f"a) País con mayor índice: {paismax} ({valormax})")
print(f"   País con menor índice (distinto de cero): {paismin} ({valormin})")

a) País con mayor índice: Kosovo (64536.0)
   País con menor índice (distinto de cero): Austria (0.5)


In [103]:
yearpromedio = pivot_table.mean(axis=0)
yearmax = yearpromedio.idxmax()
yearmaxpromedio = yearpromedio.max()
yearmin = yearpromedio.idxmin()
yearminpromedio = yearpromedio.min()

print(f"b) Año con promedio más alto: {yearmax} ({yearmaxpromedio:.2f})")
print(f"   Año con promedio más bajo: {yearmin} ({yearminpromedio:.2f})")

b) Año con promedio más alto: 2013 (449.11)
   Año con promedio más bajo: 2001 (20.03)


In [104]:
variab = pivot_table.max(axis=1) - pivot_table.replace(0, np.nan).min(axis=1)
masvariable = variab.idxmax()
valmasvariable = variab.max()

print(f"c) País con mayor variabilidad: {masvariable} (variabilidad: {valmasvariable:.2f})")

c) País con mayor variabilidad: Kosovo (variabilidad: 46098.00)


In [105]:
#d)
paisesctes = pivot_table.nunique(axis=1) == 1
listapaisesctes = pivot_table[paisesctes].index.tolist()

print(f"d) Países con índice constante: {listapaisesctes if len(listapaisesctes) > 0 else 'Ninguno'}")


d) Países con índice constante: Ninguno


In [106]:
#e)
paises_sindatos = pivot_table.sum(axis=1) == 0
lista_psindatos = pivot_table[paises_sindatos].index.tolist()

print(f"e) Países sin datos: {lista_psindatos if len(lista_psindatos) > 0 else 'Ninguno'}")

e) Países sin datos: Ninguno
